# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
#pip install scipy

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
# from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
from secrets_algo import IEX_CLOUD_API_TOKEN 

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = "AAPL"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2678839187751,
 'week52high': 187.59,
 'week52low': 134.25,
 'week52highSplitAdjustOnly': 189.25,
 'week52lowSplitAdjustOnly': 129.2,
 'week52change': 0.11050377094618558,
 'sharesOutstanding': 16515111726,
 'float': 0,
 'avg10Volume': 65237855,
 'avg30Volume': 67733332,
 'day200MovingAvg': 163.7,
 'day50MovingAvg': 164.62,
 'employees': 149941,
 'ttmEPS': 6.1,
 'ttmDividendRate': 0.9210992883010946,
 'dividendYield': 0.005751106574940721,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-26',
 'nextEarningsDate': '2022-10-12',
 'peRatio': 27.240422388219,
 'beta': 1.311799983080238,
 'maxChangePercent': 63.78936568278245,
 'year5ChangePercent': 3.439330844015471,
 'year2ChangePercent': 0.31186559925479873,
 'year1ChangePercent': 0.11803777921383474,
 'ytdChangePercent': -0.07593915589327097,
 'month6ChangePercent': -0.004800208080935868,
 'month3ChangePercent': 0.14431148610227276,
 'month1ChangePercent': 0.0819611180407295,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.11803777921383474

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([ symbol,
                        data[symbol]['price'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'            
            ], index= my_columns),
            ignore_index= True
       )
final_dataframe

/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/1341188032.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/1341188032.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/1341188032.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/1341188032.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,132.84,-0.257674,N/A
1,AAL,13.88,-0.310151,N/A
2,AAP,176.53,-0.148711,N/A
3,AAPL,167.65,0.116555,N/A
4,ABBV,140.84,0.185669,N/A
...,...,...,...,...
95,CINF,102.55,-0.159833,N/A
96,CL,79.34,0.050170,N/A
97,CLX,146.46,-0.101383,N/A
98,CMA,83.57,0.148095,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,89,CF,119.83,1.710106,N/A
1,42,APA,40.64,1.322494,N/A
2,11,ADM,92.34,0.550457,N/A
3,94,CI,289.33,0.401491,N/A
4,77,CAH,68.80,0.379001,N/A
5,54,AZO,2180.63,0.377479,N/A
6,37,ANET,124.01,0.339025,N/A
7,21,AJG,191.40,0.335439,N/A
8,92,CHRW,120.26,0.319634,N/A
9,19,AIV,9.00,0.317735,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! /nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

1e6


In [11]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,89,CF,119.83,1.710106,166
1,42,APA,40.64,1.322494,492
2,11,ADM,92.34,0.550457,216
3,94,CI,289.33,0.401491,69
4,77,CAH,68.80,0.379001,290
5,54,AZO,2180.63,0.377479,9
6,37,ANET,124.01,0.339025,161
7,21,AJG,191.40,0.335439,104
8,92,CHRW,120.26,0.319634,166
9,19,AIV,9.00,0.317735,2222


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns= hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([

                
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
            ], 
            index = hqm_columns
            ), ignore_index= True
        )

hqm_dataframe    

/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/2991633981.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/2991633981.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/2991633981.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
/var/folders/8s/_dbmw0rn6vl27xcpdbhd4g0c0000gn/T/ipykernel_18354/2991633981.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,133.26,N/A,-0.252351,N/A,-0.026175,N/A,0.052123,N/A,0.038549,N/A,N/A
1,AAL,14.09,N/A,-0.315682,N/A,-0.220875,N/A,-0.206135,N/A,0.023343,N/A,N/A
2,AAP,178.88,N/A,-0.153508,N/A,-0.148602,N/A,-0.086151,N/A,-0.052937,N/A,N/A
3,AAPL,170.27,N/A,0.119214,N/A,-0.004647,N/A,0.139656,N/A,0.083931,N/A,N/A
4,ABBV,142.54,N/A,0.186043,N/A,-0.074653,N/A,-0.086159,N/A,-0.100433,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,116.43,N/A,-0.135395,N/A,-0.087909,N/A,-0.043536,N/A,-0.044361,N/A,N/A
497,ZBH,109.79,N/A,-0.286449,N/A,-0.148798,N/A,-0.108508,N/A,-0.004943,N/A,N/A
498,ZBRA,316.77,N/A,-0.485999,N/A,-0.274327,N/A,-0.075773,N/A,-0.057985,N/A,N/A
499,ZION,58.28,N/A,0.038848,N/A,-0.196633,N/A,0.007995,N/A,0.079777,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,133.26,N/A,-0.252351,N/A,-0.026175,N/A,0.052123,N/A,0.038549,N/A,N/A
1,AAL,14.09,N/A,-0.315682,N/A,-0.220875,N/A,-0.206135,N/A,0.023343,N/A,N/A
2,AAP,178.88,N/A,-0.153508,N/A,-0.148602,N/A,-0.086151,N/A,-0.052937,N/A,N/A
3,AAPL,170.27,N/A,0.119214,N/A,-0.004647,N/A,0.139656,N/A,0.083931,N/A,N/A
4,ABBV,142.54,N/A,0.186043,N/A,-0.074653,N/A,-0.086159,N/A,-0.100433,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,116.43,N/A,-0.135395,N/A,-0.087909,N/A,-0.043536,N/A,-0.044361,N/A,N/A
497,ZBH,109.79,N/A,-0.286449,N/A,-0.148798,N/A,-0.108508,N/A,-0.004943,N/A,N/A
498,ZBRA,316.77,N/A,-0.485999,N/A,-0.274327,N/A,-0.075773,N/A,-0.057985,N/A,N/A
499,ZION,58.28,N/A,0.038848,N/A,-0.196633,N/A,0.007995,N/A,0.079777,N/A,N/A


In [14]:
hqm_dataframe[hqm_dataframe['One-Year Price Return'].isnull()]  

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
118,CTL,11.000,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
164,ETFC,50.390,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
324,MYL,16.564,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
325,NBL,8.660,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A


In [15]:
hqm_dataframe = hqm_dataframe.fillna(0)
hqm_dataframe.sort_values('One-Year Price Return')

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
385,PYPL,95.00,N/A,-0.677528,N/A,-0.170496,N/A,0.154105,N/A,0.210515,N/A,N/A
24,ALGN,262.03,N/A,-0.660883,N/A,-0.528422,N/A,-0.085164,N/A,0.002422,N/A,N/A
451,UAA,8.98,N/A,-0.635049,N/A,-0.509535,N/A,-0.141062,N/A,0.059751,N/A,N/A
450,UA,7.90,N/A,-0.629468,N/A,-0.511024,N/A,-0.150157,N/A,0.053308,N/A,N/A
207,GPS,10.10,N/A,-0.629233,N/A,-0.340032,N/A,-0.117627,N/A,0.130103,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
315,MRO,26.17,N/A,1.314782,N/A,0.207750,N/A,-0.118323,N/A,0.150967,N/A,N/A
147,DVN,75.33,N/A,1.672949,N/A,0.365162,N/A,0.002288,N/A,0.287672,N/A,N/A
89,CF,123.74,N/A,1.731213,N/A,0.504765,N/A,0.230600,N/A,0.324474,N/A,N/A
355,OXY,75.91,N/A,2.087659,N/A,0.907306,N/A,0.074597,N/A,0.190526,N/A,N/A


In [16]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,133.26,N/A,-0.252351,18.962076,-0.026175,58.682635,0.052123,76.646707,0.038549,53.093812,N/A
1,AAL,14.09,N/A,-0.315682,11.776447,-0.220875,15.568862,-0.206135,4.790419,0.023343,45.508982,N/A
2,AAP,178.88,N/A,-0.153508,36.726547,-0.148602,31.936128,-0.086151,23.153693,-0.052937,11.976048,N/A
3,AAPL,170.27,N/A,0.119214,77.844311,-0.004647,61.676647,0.139656,92.215569,0.083931,79.441118,N/A
4,ABBV,142.54,N/A,0.186043,82.634731,-0.074653,47.704591,-0.086159,22.954092,-0.100433,3.592814,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,116.43,N/A,-0.135395,40.319361,-0.087909,44.111776,-0.043536,37.52495,-0.044361,14.371257,N/A
497,ZBH,109.79,N/A,-0.286449,14.371257,-0.148798,31.736527,-0.108508,17.165669,-0.004943,27.145709,N/A
498,ZBRA,316.77,N/A,-0.485999,3.393214,-0.274327,9.780439,-0.075773,26.347305,-0.057985,10.379242,N/A
499,ZION,58.28,N/A,0.038848,67.664671,-0.196633,19.760479,0.007995,61.477046,0.079777,77.644711,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [17]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'] )
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,133.26,N/A,-0.252351,18.962076,-0.026175,58.682635,0.052123,76.646707,0.038549,53.093812,51.846307
1,AAL,14.09,N/A,-0.315682,11.776447,-0.220875,15.568862,-0.206135,4.790419,0.023343,45.508982,19.411178
2,AAP,178.88,N/A,-0.153508,36.726547,-0.148602,31.936128,-0.086151,23.153693,-0.052937,11.976048,25.948104
3,AAPL,170.27,N/A,0.119214,77.844311,-0.004647,61.676647,0.139656,92.215569,0.083931,79.441118,77.794411
4,ABBV,142.54,N/A,0.186043,82.634731,-0.074653,47.704591,-0.086159,22.954092,-0.100433,3.592814,39.221557
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,116.43,N/A,-0.135395,40.319361,-0.087909,44.111776,-0.043536,37.52495,-0.044361,14.371257,34.081836
497,ZBH,109.79,N/A,-0.286449,14.371257,-0.148798,31.736527,-0.108508,17.165669,-0.004943,27.145709,22.60479
498,ZBRA,316.77,N/A,-0.485999,3.393214,-0.274327,9.780439,-0.075773,26.347305,-0.057985,10.379242,12.47505
499,ZION,58.28,N/A,0.038848,67.664671,-0.196633,19.760479,0.007995,61.477046,0.079777,77.644711,56.636727


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [18]:
hqm_dataframe.sort_values('HQM Score', ascending= False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace= True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CF,123.74,N/A,1.731213,99.600798,0.504765,99.001996,0.230600,98.602794,0.324474,99.800399,99.251497
1,HRB,48.45,N/A,0.925175,97.804391,0.938233,100.0,0.369640,99.800399,0.202936,96.806387,98.602794
2,COG,22.94,N/A,0.350065,91.616766,0.184615,91.217565,0.268368,99.001996,0.259856,98.802395,95.159681
3,AIV,9.00,N/A,0.325179,90.219561,0.242228,95.808383,0.453204,100.0,0.172859,93.413174,94.860279
4,UNM,39.34,N/A,0.542326,95.808383,0.399765,98.602794,0.095654,87.42515,0.207500,97.205589,94.760479
5,CAH,67.80,N/A,0.373639,93.013972,0.279364,96.806387,0.178095,95.209581,0.177695,93.812375,94.710579
6,ALB,300.21,N/A,0.265406,87.824351,0.509291,99.201597,0.135464,91.816367,0.293032,99.401198,94.560878
7,OXY,75.91,N/A,2.087659,99.800399,0.907306,99.800399,0.074597,82.43513,0.190526,95.409182,94.361277
8,GWW,584.52,N/A,0.350550,91.816367,0.204973,93.213573,0.190721,96.407186,0.187016,95.00998,94.111776
9,LB,80.77,N/A,2.321580,100.0,0.847054,99.600798,0.216730,98.003992,0.080679,78.043912,93.912176


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

In [20]:
position_size = float(portfolio_size)/ len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CF,123.74,161,1.731213,99.600798,0.504765,99.001996,0.230600,98.602794,0.324474,99.800399,99.251497
1,HRB,48.45,412,0.925175,97.804391,0.938233,100.0,0.369640,99.800399,0.202936,96.806387,98.602794
2,COG,22.94,871,0.350065,91.616766,0.184615,91.217565,0.268368,99.001996,0.259856,98.802395,95.159681
3,AIV,9.00,2222,0.325179,90.219561,0.242228,95.808383,0.453204,100.0,0.172859,93.413174,94.860279
4,UNM,39.34,508,0.542326,95.808383,0.399765,98.602794,0.095654,87.42515,0.207500,97.205589,94.760479
5,CAH,67.80,294,0.373639,93.013972,0.279364,96.806387,0.178095,95.209581,0.177695,93.812375,94.710579
6,ALB,300.21,66,0.265406,87.824351,0.509291,99.201597,0.135464,91.816367,0.293032,99.401198,94.560878
7,OXY,75.91,263,2.087659,99.800399,0.907306,99.800399,0.074597,82.43513,0.190526,95.409182,94.361277
8,GWW,584.52,34,0.350550,91.816367,0.204973,93.213573,0.190721,96.407186,0.187016,95.00998,94.111776
9,LB,80.77,247,2.321580,100.0,0.847054,99.600798,0.216730,98.003992,0.080679,78.043912,93.912176


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [48]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name= "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [49]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [50]:
columns_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', integer_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', integer_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', integer_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', integer_template],
    'L': ['HQM Score', integer_template] 
}

for column in columns_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, columns_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', columns_formats[column][0], columns_formats[column][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [51]:
writer.save()